In [1]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

In [3]:
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
os.environ["LANGSMITH_TRACING"] = "true"

In [63]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [68]:
messages = [
    (
        "system",
        "You are a special kind of agent that when i give you an question, you will get answer for that question by solve that in step-by-step.",
    ),
    ("human", "How many `r` letters in the word `strawberry`"),
]
ai_msg = llm.invoke(messages)

In [69]:
ai_msg

AIMessage(content="To find the number of `r` letters in the word `strawberry`, let's break it down step by step:\n\n1. **Write down the word**: The word is `strawberry`.\n2. **Separate the letters**: Separate each letter in the word to count them individually: `s-t-r-a-w-b-e-r-r-y`.\n3. **Count the `r` letters**: Now, count how many times the letter `r` appears in the separated letters.\n   - The first `r` is after `t`.\n   - The second `r` is after `e`.\n   - There is another `r` right after the second `r`.\n4. **Conclusion**: There are 3 `r` letters in the word `strawberry`.\n\nTherefore, the answer is: **3** `r` letters in the word `strawberry`.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 182, 'prompt_tokens': 79, 'total_tokens': 261, 'completion_time': 0.968598425, 'prompt_time': 0.019409033, 'queue_time': 0.31940539599999995, 'total_time': 0.988007458}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c0cfa69934', 'finish_reaso

In [70]:
from IPython.display import display, Markdown

# Display the response
display(Markdown(ai_msg.content))

To find the number of `r` letters in the word `strawberry`, let's break it down step by step:

1. **Write down the word**: The word is `strawberry`.
2. **Separate the letters**: Separate each letter in the word to count them individually: `s-t-r-a-w-b-e-r-r-y`.
3. **Count the `r` letters**: Now, count how many times the letter `r` appears in the separated letters.
   - The first `r` is after `t`.
   - The second `r` is after `e`.
   - There is another `r` right after the second `r`.
4. **Conclusion**: There are 3 `r` letters in the word `strawberry`.

Therefore, the answer is: **3** `r` letters in the word `strawberry`.